<img src=".images/logosnb.png" alt="Banner" style="width: 1100px;"/>

<div style='color: #690027;' markdown="1">
    <h1>CLASSIFICATIE STOMATA OP BEZONDE EN BESCHADUWDE BLADEREN</h1> 
</div>

<div class="alert alert-box alert-success">
In deze notebook zal je bezonde en beschaduwde bladeren van elkaar scheiden. De twee klassen zijn bij benadering lineair scheidbaar. 
</div>

Krappa of crabwood is een snel groeiende boomsoort die veelvuldig voorkomt in het Amazonegebied. Volwassen exemplaren kunnen een diameter hebben van meer dan een meter en kunnen meer dan 40 meter hoog zijn. Het hout van hoge kwaliteit wordt gebruikt voor het maken van meubelen, vloeren, masten... Uit de schors wordt een koorstwerend middel gehaald. Uit de zaden produceert men een olie voor medicinale toepassingen, waaronder de behandeling van huidziekten en tetanos, en als afweermiddel voor insecten. 

<table><tr>
<td> <img src=".images/andirobaamazonica.jpg" alt="Drawing" style="width: 200px;"/></td>
<td> <img src=".images/crabwoodtree.jpg" alt="Drawing" style="width: 236px;"/> </td>
</tr></table>

<center>
Foto's: Mauroguanandi [Public domain] [2] en P. S. Sena [CC BY-SA 4.0] [3].
</center>

Omdat sommige klimaatmodellen een stijging van de temperatuur en een vermindering in regenval voorspellen in de komende decennia, is het belangrijk om te weten hoe deze bomen zich aanpassen aan veranderende omstandigheden. <br>
Wetenschappers Camargo en Marenco deden onderzoek in het Amazonewoud [1].<br>
Naast de invloed van seizoensgebonden regenval, bekeken ze ook stomatale kenmerken van bladeren onder bezonde en onder beschaduwde condities.<br> Hiervoor werden een aantal planten, opgekweekt in de schaduw, verplaatst naar vol zonlicht gedurende 60 dagen. Een andere groep planten werd in de schaduw gehouden. <br>De kenmerken van de stomata werden opgemeten op afdrukken van de bladeren gemaakt met transparante nagellak. 

### Nodige modules importeren

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LogisticRegression

from matplotlib import animation
from IPython.display import HTML

<div style='color: #690027;' markdown="1">
    <h2>1. Inlezen van de data</h2> 
</div>

Lees met de module `pandas` de dataset in.

In [ ]:
stomata = pd.read_csv(".data/schaduwzon.dat", header="infer")  # in te lezen tabel heeft een hoofding

<div style='color: #690027;' markdown="1">
    <h2>2. Tonen van de ingelezen data</h2> 
</div>

<div style='color: #690027;' markdown="1">
    <h3>2.1 Tabel met de data</h2> 
</div>

Kijk de gegevens in. 

In [ ]:
stomata

Welke gegevens zijn kenmerken? <br> Welk gegeven is het label? <br> 
Deze gegevens kunnen worden gevisualiseerd met een puntenwolk. Welke matrices heb je daarvoor nodig? 

Antwoord:
De plantensoort is overal dezelfde: Carapa. <br>
De kenmerken zijn de stomatale dichtheid en de stomatale grootte. <br>
Het aantal monsters is 50.<br>
Het label is het milieu waarin het monster werd geplukt: zon of schaduw.<br>
Om de puntenwolk weer te geven, heb je twee matrices nodig met dimensie 50x1. 

De onderzoekers zetten de stomatale dichtheid uit tegenover de stomatale lengte.<br> Ga op dezelfde manier te werk.

<div style='color: #690027;' markdown="1">
    <h3>2.2 De data weergeven in puntenwolk</h2> 
</div>

In [ ]:
x1 = stomata["stomatale lengte"]          # kenmerk: lengte
x2 = stomata["stomatale dichtheid"]       # kenmerk: dichtheid

In [ ]:
x1 = np.array(x1)          # kenmerk: lengte
x2 = np.array(x2)          # kenmerk: dichtheid

In [ ]:
# dichtheid t.o.v. lengte
plt.figure()

plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn de volgende 25
           
plt.title("Carapa")
plt.xlabel("stomatale lengte (micron)")
plt.ylabel("stomatale densiteit (per mm²)")
plt.legend(loc="lower left")

plt.show()

<div style='color: #690027;' markdown="1">
    <h2>3. Standaardiseren</h2> 
</div>

<div style='color: #690027;' markdown="1">
    <h3>3.1 Lineair scheidbaar?</h3> 
</div>

Er zijn twee groepen te onderscheiden. Ze zijn op enkele punten na lineair scheidbaar.

De grootte-orde van deze gegevens is sterk verschillend. De gegevens moeten gestandaardiseerd worden. 

<div style='color: #690027;' markdown="1">
    <h3>3.2 Standaardiseren</h3> 
</div>

<div class="alert alert-block alert-warning">
Meer uitleg over het belang van standaardiseren vind je in de notebook 'Standaardiseren'.
</div>

In [ ]:
x1_gem = np.mean(x1)
x1_std = np.std(x1)
x2_gem = np.mean(x2)
x2_std = np.std(x2)
x1 = (x1 - x1_gem) / x1_std
x2 = (x2 - x2_gem) / x2_std

In [ ]:
# dichtheid t.o.v. lengte
plt.figure()

plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn de volgende 25
           
plt.title("Carapa")
plt.xlabel("gestandaardiseerde stomatale lengte (micron)")
plt.ylabel("gestandaardiseerde stomatale densiteit (per mm²)")
plt.legend(loc="lower left")

plt.show()

<div style='color: #690027;' markdown="1">
    <h2>4. Classificatie met Perceptron</h2> 
</div>

<div style='color: #690027;' markdown="1">
    <h3>4.1 Geannoteerde data</h2> 
</div>

Het ML-systeem zal machinaal leren uit de 50 gelabelde voorbeelden.<br> 
Lees de labels in.

In [ ]:
y = stomata["milieu"]            # labels: tweede kolom van de oorspronkelijke tabel
y = np.array(y)
print(y)

In [ ]:
y = np.where(y == "zon", 1, 0)     # labels numeriek maken, zon:1, schaduw:0
print(y)

In [ ]:
X = np.stack((x1, x2), axis = 1)    # omzetten naar gewenste formaat

<div style='color: #690027;' markdown="1">
    <h3>4.2 Perceptron</h2> 
</div>

<div class="alert alert-box alert-info">
Als twee klassen lineair scheidbaar zijn, kan men een rechte vinden die beide klassen scheidt. Men kan de vergelijking van de scheidingslijn opschrijven in de vorm $ax+by+c=0$. Voor elk punt $(x_{1}, y_{1})$ in de ene klasse is dan $ax_{1}+by_{1}+c \geq 0$ en voor elk punt $(x_{2}, y_{2})$ in de andere klasse is dan $ax_{2} +by_{2}+c < 0$. <br> 
Zolang dit niet voldaan is, moeten de coëfficiënten worden aangepast.<br>
De trainingset met bijhorende labels wordt enkele keren doorlopen. Voor elk punt worden de coëfficiënten aangepast indien nodig.
</div>

Er wordt een willekeurige rechte gekozen die de twee soorten bladeren zou moeten scheiden. Dit gebeurt door de coëfficiënten in de vergelijking van de rechte willekeurig te kiezen. Beide kanten van de scheidingslijn bepalen een andere klasse. <br>Met systeem wordt getraind met de trainingset en de gegeven labels. Voor elk punt van de trainingset wordt nagegaan of het punt aan de juiste kant van de scheidingslijn ligt. Bij een punt die niet aan de juiste kant van de scheidingslijn ligt, worden de coëfficiënten in de vergelijking van de rechte aangepast. <br>
De volledige trainingset wordt een aantal keer doorlopen. Het systeem leert gedurende deze 'pogingen' of *epochs*.

In [ ]:
def grafiek(coeff_x1, coeff_x2, cte):
        """Plot scheidingsrechte ('decision boundary') en geeft vergelijking ervan."""
        # stomatale densiteit t.o.v. lengte van stomata
        plt.figure()
        
        plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25 (label 1)
        plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn de volgende 25 (label 0)
        x = np.linspace(-1.5, 1.5, 10)
        y_r = -coeff_x1/coeff_x2 * x - cte/coeff_x2
        print("De grens is een rechte met vgl.", coeff_x1, "* x1 +", coeff_x2, "* x2 +", cte, "= 0")
        plt.plot(x, y_r, color="black")
        
        plt.title("Classificatie Carapa")
        plt.xlabel("gestandaardiseerde stomatale lengte (micron)")
        plt.ylabel("gestandaardiseerde stomatale densiteit (per mm²)")
        plt.legend(loc="lower left")
        
        plt.show()

class Perceptron(object):
    """Perceptron classifier.""" 
    
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        """self heeft drie parameters: leersnelheid, aantal pogingen, willekeurigheid."""
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        """Fit training data."""
        rgen = np.random.RandomState(self.random_state)
        # kolommatrix van de gewichten ('weights')
        # willekeurig gegenereerd uit normale verdeling met gemiddelde 0 en standaardafwijking 0.01
        # aantal gewichten is aantal kenmerken in X plus 1 (+1 voor bias)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1]+1)     # gewichtenmatrix die 3 gewichten bevat 
        print("Initiële willekeurige gewichten:", self.w_)
        self.errors_ = []    # foutenlijst
       
        # plot grafiek met initiële scheidingsrechte
        print("Initiële willekeurige rechte:")
        grafiek(self.w_[1], self.w_[2], self.w_[0])
        gewichtenlijst = np.array([self.w_])
                
        # gewichten punt per punt aanpassen, gebaseerd op feedback van de verschillende pogingen        
        for _ in range(self.n_iter):
            print("epoch =", _)
            errors = 0
            teller = 0
            for x, label in zip(X, y):            # x is datapunt, y overeenkomstig label
                print("teller =", teller)         # tel punten, het zijn er acht
                print("punt:", x, "\tlabel:", label)
                gegiste_klasse = self.predict(x)
                print("gegiste klasse =", gegiste_klasse)
                # aanpassing nagaan voor dit punt
                update = self.eta * (label - gegiste_klasse)     # als update = 0, juiste klasse, geen aanpassing nodig
                print("update =", update)
                # grafiek en gewichten eventueel aanpassen na dit punt
                if update !=0:
                    self.w_[1:] += update *x
                    self.w_[0] += update
                    errors += update
                    print("gewichten =", self.w_) # bepalen voorlopige 'decision boundary'
                    gewichtenlijst = np.append(gewichtenlijst, [self.w_], axis =0)
                teller += 1
            self.errors_.append(errors)           # na alle punten, totale fout toevoegen aan foutenlijst
            print("foutenlijst =", self.errors_)          
        return self, gewichtenlijst               # geeft lijst gewichtenmatrices terug
    
    def net_input(self, x):      # punt invullen in de voorlopige scheidingsrechte
        """Berekenen van z = lineaire combinatie van de  inputs inclusief bias en de weights voor elke gegeven punt."""
        return np.dot(x, self.w_[1:]) + self.w_[0]
    
    def predict(self, x):
        """Gist klasse."""
        print("punt ingevuld in vergelijking rechte:", self.net_input(x))
        klasse = np.where(self.net_input(x) >=0, 1, 0)
        return klasse
    

In [ ]:
# perceptron, leersnelheid 0.0001 en 20 pogingen
ppn = Perceptron(eta=0.0001, n_iter=20)
gewichtenlijst = ppn.fit(X,y)[1]
print("Gewichtenlijst =", gewichtenlijst)

In [ ]:
# animatie

xcoord = np.linspace(-1.5, 1.5, 10)

ycoord = []
for w in gewichtenlijst:
    y_r = -w[1]/w[2] * xcoord - w[0]/w[2]
    ycoord.append(y_r)
ycoord = np.array(ycoord)    # type casting

fig, ax = plt.subplots()
line, = ax.plot(xcoord, ycoord[0])

plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="zon")      # zon zijn eerste 25 (label 1)
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="schaduw")   # schaduw zijn de volgende 25 (label 0)

ax.axis([-2,2,-2,2])

def animate(i):
    line.set_ydata(ycoord[i])  # update de vergelijking van de rechte  
    return line,

plt.close()  # om voorlopig plot-venster te sluiten, enkel animatiescherm nodig

anim = animation.FuncAnimation(fig, animate, interval=1000, repeat=False, frames=len(ycoord))

HTML(anim.to_jshtml())

Mooi resultaat! Maar nog niet optimaal. 
### Opdracht 4.2
Wellicht bieden meer iteraties nog een beter resultaat. Probeer eens uit.

<div class="alert alert-block alert-info">
Omdat de klassen niet lineair scheidbaar zijn, zal het Perceptron er natuurlijk niet in slagen de fout op nul te krijgen. Door de leersnelheid en het aantal epochs zo goed mogelijke te kiezen, kan je een zo goed mogelijke scheiding proberen bekomen.<br>
Bij niet-lineair scheidbare klassen zal men daarom in machinaal leren geen Perceptron gebruiken, maar de klassen optimaal proberen scheiden op een andere manier: met gradient descent voor de aanpassingen en binary cross entropy om de fout te bepalen.
</div>

<img src=".images/cclic.png" alt="Banner" align="left" style="width:100px;"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI op School</a>, van F. wyffels & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>.